In [1]:
import GPyOpt
import GPy
import numpy as np
import pandas as pd
import re
import json
import sys

sys.path.append("../../devel/")

from binomial_optimization import optimization_step, get_new_point, fidelity_decision

In [2]:
N_EVENTS = 100
N_EVENTS_high = 100
treshold_proba = 0.5
method = 'gp'

# Optimization space

In [3]:
min_dist = 3.6
radius = 2

space = [{'name': 'pitch', 'type': 'continuous', 'domain': (min_dist, min_dist)},\
         {'name': 'yoffset_layer', 'type': 'continuous', 'domain': (min_dist/2, min_dist)},\
         {'name': 'yoffset_plane', 'type': 'continuous', 'domain': (min_dist*0.25, min_dist*1.25)},\
         {'name': 'zshift_layer', 'type': 'continuous', 'domain': (1, 12)},\
         {'name': 'zshift_plane', 'type': 'continuous', 'domain': (1, 12)},\
         {'name': 'zshift_view', 'type': 'continuous', 'domain': (10, 12)},\
         {'name': 'alpha', 'type': 'continuous', 'domain': (5, 15)}]

constraints = [{'name': 'constr_1', 'constrain': '-(x[:,0]-x[:,1])**2-x[:,3]**2+'+str(radius)+'**2'},\
               {'name': 'constr_2', 'constrain': '-(x[:,1]-x[:,2])**2-(x[:,3]-x[:,4])**2+'+str(radius)+'**2'},\
               {'name': 'constr_3', 'constrain': 'x[:,3]+x[:,4]+'+str(radius)+'-x[:,5]'},
               {'name': 'constr_4', 'constrain': 'x[:,3]+'+str(radius)+'-x[:,4]'}]

lower_bounds = [min_dist, min_dist/2, min_dist*0.25, 1, 1, 10, 5]
upper_bounds = [min_dist, min_dist, min_dist*1.25, 12, 12, 12, 15]
constraints_opt = [{'type': 'ineq', 'fun': lambda t: -(t[0]-t[1])**2-t[3]**2+radius**2},
                   {'type': 'ineq', 'fun': lambda t: -(t[1]-t[2])**2-(t[3]-t[4])**2+radius**2},
                   {'type': 'ineq', 'fun': lambda t: t[3]+t[4]+radius-t[5]},
                   {'type': 'ineq', 'fun': lambda t: t[3]+radius-t[4]}]

In [4]:
feasible_region = GPyOpt.Design_space(space=space, constraints=constraints)

np.random.seed(7)

# Import skygrid client

In [5]:
import time
import json

from disneylandClient import (
    new_client,
    Job,
    RequestWithId,
)

In [6]:
STATUS_IN_PROCESS = set([
    Job.PENDING,
    Job.PULLED,
    Job.RUNNING,
])
STATUS_FINAL = set([
    Job.COMPLETED,
    Job.FAILED,
])

In [7]:
def return_descriptor(point, n_events=N_EVENTS):
    
    pitch, yoffset_layer, yoffset_plane, zshift_layer, zshift_plane, zshift_view, alpha = point
    
    cmd = "/opt/disney-run.sh python /opt/objective.py --pitch "+str(pitch)+" --yoffset_layer "+str(yoffset_layer)+\
        " --yoffset_plane "+str(yoffset_plane)+" --zshift_layer "+str(zshift_layer)+" --zshift_plane "+\
        str(zshift_plane)+" --zshift_view "+str(zshift_view)+" --alpha "+str(int(alpha))+\
        " --nEvents "+str(n_events)+" --method FH"

    descriptor = {
        "input": [],

        "container": {
            "workdir": "",
            "name": "oleg94/ship_metric:latest",
            "cpu_needed": 1,
            "max_memoryMB": 4096,
            "min_memoryMB": 1024,
            "cmd": cmd,
        },

        "required_outputs": {
            "output_uri": "none:",
            "file_contents": [
                {"file": "output.txt", "to_variable": "out"}
            ]
        }
    }
    
    return descriptor

# Initial design

In [8]:
n_estimators = 20
n_initial_design = 20
initial_design = GPyOpt.experiment_design.initial_design('random', feasible_region, n_initial_design)
initial_objective = np.zeros(n_initial_design)
stub = new_client()
jobs = []

In [9]:
init_d_i = 0

for epoch in range(n_initial_design // n_estimators):
    
    print("EPOCH #"+str(epoch)+" started.")
    
    epoch_jobs = [0] * n_estimators
    
    for k in range(n_estimators):
        descriptor = return_descriptor(initial_design[init_d_i])
        init_d_i += 1
        epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
        epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
        print(k, " pushed")
    
    prev_number_of_finished_jobs = 0
    prev_number_of_running_jobs = 0
    prev_number_of_pending_jobs = 0
    
    while True:
        for k in range(n_estimators):
            epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))
        
        number_of_finished_jobs = 0
        number_of_running_jobs = 0
        number_of_pending_jobs = 0
        for k in range(n_estimators):
            if epoch_jobs[k].status in STATUS_FINAL:
                number_of_finished_jobs += 1
            if epoch_jobs[k].status == Job.PENDING:
                number_of_pending_jobs += 1
            if epoch_jobs[k].status == Job.RUNNING:
                number_of_running_jobs += 1
                
        if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
            print("Finished jobs: "+str(number_of_finished_jobs)+\
                  " Running jobs: "+str(number_of_running_jobs)+\
                  " Pending jobs: "+str(number_of_pending_jobs))
            prev_number_of_finished_jobs = number_of_finished_jobs
            prev_number_of_running_jobs = number_of_running_jobs
            prev_number_of_pending_jobs = number_of_pending_jobs
            
        if number_of_finished_jobs == n_estimators:
            break
        time.sleep(10)
    
    jobs += epoch_jobs

EPOCH #0 started.
0  pushed
1  pushed
2  pushed
3  pushed
4  pushed
5  pushed
6  pushed
7  pushed
8  pushed
9  pushed
10  pushed
11  pushed
12  pushed
13  pushed
14  pushed
15  pushed
16  pushed
17  pushed
18  pushed
19  pushed
Finished jobs: 0 Running jobs: 0 Pending jobs: 20
Finished jobs: 0 Running jobs: 20 Pending jobs: 0
Finished jobs: 3 Running jobs: 17 Pending jobs: 0
Finished jobs: 8 Running jobs: 12 Pending jobs: 0
Finished jobs: 16 Running jobs: 4 Pending jobs: 0
Finished jobs: 17 Running jobs: 3 Pending jobs: 0
Finished jobs: 20 Running jobs: 0 Pending jobs: 0


In [10]:
df_init_design = pd.DataFrame(initial_design, columns=['pitch', 'yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane', 'zshift_view', 'alpha'])
df_init_design['reconstructible'] = [float(json.loads(re.sub(r"\\", '', job.output[15:-2]))['reconstructible']) if re.sub(r"\\", '', job.output[15:-2]) else np.nan for job in jobs]
df_init_design['reco_passed_no_clones'] = [float(json.loads(re.sub(r"\\", '', job.output[15:-2]))['reco_passed_no_clones']) if re.sub(r"\\", '', job.output[15:-2]) else np.nan for job in jobs]
df_init_design['total'] = N_EVENTS

In [11]:
df_init_design

,pitch,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,zshift_view,alpha,reconstructible,reco_passed_no_clones,total
0,3.6,2.214545,3.266638,1.584416,5.929327,11.625659,6.695237,35.0,33.0,100
1,3.6,2.609847,3.680054,2.711983,5.939211,11.758370,5.102107,29.0,17.0,100
2,3.6,2.061110,1.358748,1.748686,7.136709,11.894160,11.513498,33.0,32.0,100
3,3.6,3.366531,2.982320,2.805552,5.403728,11.062234,11.194571,27.0,14.0,100
4,3.6,3.180110,3.506603,2.039578,4.040148,10.678221,12.067209,28.0,14.0,100
5,3.6,2.304825,1.528151,1.728609,6.859235,11.142899,12.904034,36.0,34.0,100
6,3.6,2.893504,3.947823,3.009093,5.146496,10.655699,7.483593,25.0,19.0,100
7,3.6,3.371889,2.904746,2.266296,5.458057,11.082347,5.775261,43.0,24.0,100
8,3.6,3.474698,3.230812,2.476511,5.975230,10.676367,13.869667,35.0,19.0,100
9,3.6,3.107183,1.419664,2.509852,6.204344,11.768348,7.367284,36.0,32.0,100


In [13]:
df_init_design.to_csv('data/initial_observations_'+str(N_EVENTS)+'.csv', index=False)
df_init_design.to_csv('data/observations_'+str(N_EVENTS)+'_gp.csv', index=False)
df_init_design.to_csv('data/observations_'+str(N_EVENTS)+'_ggpm.csv', index=False)
df_init_design.to_csv('data/observations_'+str(N_EVENTS)+'_f.csv', index=False)

# Main part of optimization

In [8]:
def convertor(disney_output):
    
    return float(json.loads(re.sub(r"\\", '', disney_output[15:-2]))['reconstructible']), float(json.loads(re.sub(r"\\", '', disney_output[15:-2]))['reco_passed_no_clones'])

def simulate(point, n_events=N_EVENTS):
    
    descriptor = return_descriptor(point, n_events)
    job = stub.CreateJob(Job(input=json.dumps(descriptor), kind="docker")) 
    while True:
        job = stub.GetJob(RequestWithId(id=job.id))
        
        if job.status in STATUS_FINAL:
            break
        time.sleep(10)
    
    return convertor(job.output)

In [9]:
observed_points = pd.read_csv('data/observations_'+str(N_EVENTS)+'_'+method+'.csv')

stub = new_client()

kernel = GPy.kern.RBF(len(space))

n_iteration = 500

In [ ]:
X = observed_points.drop(['reconstructible', 'reco_passed_no_clones'], axis=1).values
Y = observed_points[['reco_passed_no_clones']].values
trials = observed_points[['reconstructible']].values

for i in range(n_iteration):
    
    new_x = None
    
    if method=='gp':
        model = GPy.models.GPRegression(X, Y/trials, kernel)
        
        model.optimize_restarts(num_restarts=10, verbose=False)
        new_x, criterion_value = get_new_point(model, data=(X, Y/trials),
                                               lower_bounds=lower_bounds, upper_bounds=upper_bounds, method='gaussian',
                                               constraints=constraints_opt, optimization_method='SLSQP')
        
    elif method=='ggpm':
        binomial = GPy.likelihoods.Binomial()
        model = GPy.core.GP(X, Y, kernel=kernel, 
                            Y_metadata={'trials': trials},
                            inference_method=GPy.inference.latent_function_inference.laplace.Laplace(),
                            likelihood=binomial)
        
        model.optimize_restarts(num_restarts=10, verbose=False)
        new_x, criterion_value = get_new_point(model, data=(X, Y),
                                               lower_bounds=lower_bounds, upper_bounds=upper_bounds, method='laplace',
                                               constraints=constraints_opt, optimization_method='SLSQP')
    
    print('Point calculated.')
    reconstructible, passed_no_clones = simulate(new_x, N_EVENTS)
    
    print('Value got.')
    if (N_EVENTS_high >= N_EVENTS+1) and (method == 'ggpm'):

        if fidelity_decision(reconstructible, passed_no_clones, 
                             model.likelihood.gp_link.transf(np.min(model._raw_predict(X)[0])), 
                             treshold_proba):

            reconstructible_additional, passed_no_clones_additional = simulate(new_x, N_EVENTS_high-N_EVENTS)

            reconstructible += reconstructible_additional
            passed_no_clones += passed_no_clones_additional
            
    observed_points.loc[len(observed_points)] = list(new_x) + [reconstructible, passed_no_clones]
    observed_points.to_csv('data/observations_'+str(N_EVENTS)+'_'+method+'.csv', index=False)
    
    print('Point saved.')

Point calculated.
Value got.
Point saved.
Point calculated.
Value got.
Point saved.
Point calculated.
Value got.
Point saved.
Value got.
Point saved.
Point calculated.
Value got.
Point saved.
Point calculated.
Value got.
Point saved.
Point calculated.
